In [1]:
!pip install fastapi nest_asyncio uvicorn pyngrok diffusers transformers torch accelerate python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
!ngrok config add-authtoken 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
# Cell 3 – imports, lazy pipeline, background job, API
import os, uuid, torch, nest_asyncio, uvicorn
from fastapi import FastAPI, UploadFile, File, BackgroundTasks, HTTPException
from fastapi.responses import StreamingResponse
from fastapi.staticfiles import StaticFiles
from threading import Thread
from pyngrok import ngrok
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

app  = FastAPI(title="Colab B – Video Worker")
jobs = {}                      # job_id ➜ dict(status|path|error)

# ---------- lazy pipeline ----------
pipe = None
def get_pipe():
    global pipe
    if pipe is None:
        pipe = StableVideoDiffusionPipeline.from_pretrained(
            "stabilityai/stable-video-diffusion-img2vid-xt",
            torch_dtype=torch.float16,
            variant="fp16"
        )
        pipe.enable_model_cpu_offload()
    return pipe

# ---------- background worker ----------
def run_job(job_id: str, img_path: str):
    try:
        img  = load_image(img_path).resize((1024, 576))
        frames = get_pipe()(img,
                            decode_chunk_size=8,
                            generator=torch.manual_seed(42)).frames[0]
        out_dir = "video_outputs"; os.makedirs(out_dir, exist_ok=True)
        mp4_path = f"{out_dir}/{job_id}.mp4"
        export_to_video(frames, mp4_path, fps=14)
        jobs[job_id] = {"status": "done", "path": mp4_path}
    except Exception as e:
        jobs[job_id] = {"status": "error", "error": str(e)}
# ---------- API ----------
@app.post("/enqueue")
async def enqueue(file: UploadFile = File(...), bg: BackgroundTasks = BackgroundTasks()):
    job_id   = str(uuid.uuid4())
    img_dir  = "received_images"; os.makedirs(img_dir, exist_ok=True)
    img_path = f"{img_dir}/{job_id}.png"
    with open(img_path, "wb") as f:
        f.write(await file.read())

    jobs[job_id] = {"status": "processing"}
    bg.add_task(run_job, job_id, img_path)
    return {"job_id": job_id}

@app.get("/result/{job_id}")
async def result(job_id: str):
    job = jobs.get(job_id)
    if not job:
        raise HTTPException(status_code=404, detail="unknown job_id")
    if job["status"] == "processing":
        return {"status": "processing"}
    if job["status"] == "error":
        return job
    # status == done  ➜  stream mp4
    return StreamingResponse(open(job["path"], "rb"),
                             media_type="video/mp4")

# ---------- serve static (optional for manual download) ----------
os.makedirs("video_outputs", exist_ok=True)
app.mount("/static", StaticFiles(directory="video_outputs"), name="static")


In [4]:
# Cell 4 – expose HTTP‑only ngrok tunnel & run server
tunnel = ngrok.connect(8000, "http", bind_tls=False)   # HTTP, no TLS
BASE_B = tunnel.public_url             # e.g. "http://abcd-12-34-56.ngrok-free.app"
print("🚀 Colab B HTTP URL:", BASE_B)

nest_asyncio.apply()
Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=8000),
       daemon=True).start()


🚀 Colab B HTTP URL: http://af1b-34-126-169-91.ngrok-free.app


In [5]:
%%javascript
function keepAlive() {
  setInterval(() => {
    google.colab.kernel.invokeFunction('notebook.ping', [], {});
    console.log("⏳ Keeping Colab alive...");
  }, 60000);  // every 60 seconds
}
keepAlive();

<IPython.core.display.Javascript object>